In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram
import meshcat

# Build plant

In [4]:

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

simple_collision = True
directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()
# visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, 
#                                       delete_prefix_on_load=False)

# diagram = builder.Build()
# visualizer.load()


# Setup meshcat visualization

In [5]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


# Build Certified Iris Region Object

In [6]:
iris_kwargs = {
    'iris_starting_ellipse_vol': 1e-5,
    'iris_plane_pullback': 1e-4,
    'iris_max_faces': -1
}

In [7]:
iris_generator = CertifiedIrisRegionGenerator(visualizer.diagram, plant, scene_graph, **iris_kwargs)

In [8]:
seed_points = np.tan(np.array([
                        [0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]      # out of shelf 1
                        ])    
                        /2)

regions, ellipses = iris_generator.iris_in_rational_space(seed_points)


0
snopt_example=[ 0.35529068 -0.22984731  0.02559963], growth = 0.05990553086765089
snopt_example=[ 0.28673366 -0.30128467  0.33402563], growth = 0.09485392273531586
snopt_example=[ 0.28776043 -0.30045108  0.32725976], growth = 0.09339195804243797
terminating because a required containment point would have not been contained
Time:   1.14 	Volume:  21.77 	Center: [1.45102197e-09 1.45111101e-09 1.45170387e-09]
0
snopt_example=[ 0.35529266 -0.22984424  0.02559962], growth = 0.059905530783997234
snopt_example=[ 0.30804713 -0.27858144  0.24207296], growth = 0.07703332294756865
terminating because a required containment point would have not been contained
Time:   0.73 	Volume:  21.77 	Center: [1.45102197e-09 1.45111101e-09 1.45170387e-09]
snopt_example=[ 0.30547805 -0.5206824   0.30020754], growth = 354169277.0183215
snopt_example=[ 0.19754042 -0.45436384  0.26846798], growth = 33224401.492606163
snopt_example=[ 0.19598982 -0.45413508  0.27106353], growth = 32414169.416864872
0
snopt_example

In [9]:
trajectory_start = np.tan(np.array([-1, -2, 0.5])/2) # seed_points[1,:]
trajectory_end = np.tan(np.array([0.25, -1.6, -0.25])/2)  #seed_points[3,:]

In [10]:
# plot regions and collision constraint
if do_viz:
    visualizer.plot_regions(regions, ellipses)
    visualizer.plot_seedpoints(seed_points)
    visualizer.visualize_collision_constraint(N = 50)
    

## Certify the regions

In [11]:
iris_generator.initalize_certifier(plane_order = 1, strict_pos_tol = 1e-4)

Time to initialize region program: 0.7432
time to create Region Certification Problem: : 0.4484
Time to initialize region program: 0.7873
time to create Region Certification Problem: : 0.4405
Time to initialize region program: 0.9503
time to create Region Certification Problem: : 1.0617
Time to initialize region program: 1.1037
time to create Region Certification Problem: : 1.7846
Time to initialize region program: 0.8971
time to create Region Certification Problem: : 0.7871
Time to initialize region program: 0.8642
time to create Region Certification Problem: : 0.6585


{<pydrake.geometry.optimization.HPolyhedron at 0x7fdd580c1830>: <certified_iris_generator.RegionCertificationProblem at 0x7fdd3f8c9ee0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7fdd3f7d6e70>: <certified_iris_generator.RegionCertificationProblem at 0x7fdd5807b2e0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7fdd3f8c0130>: <certified_iris_generator.RegionCertificationProblem at 0x7fdd3f7d9040>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7fdd580c1bf0>: <certified_iris_generator.RegionCertificationProblem at 0x7fdd3f7d91c0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7fdd3f7d5a70>: <certified_iris_generator.RegionCertificationProblem at 0x7fdd3f7d9280>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7fdd3f7ce670>: <certified_iris_generator.RegionCertificationProblem at 0x7fdd3f7d9100>}

In [12]:
do_linesearch_cert = True
if do_linesearch_cert:
    iris_generator.certify_and_adjust_regions_by_linesearch(1e-5)

Starting Region 1/6
min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.8660254  1.65860451 0.10832327 0.8660254  0.07344629 1.62372753]
max_eps = [1.73205081 3.31720903 0.21664655 1.73205081 0.14689259 3.24745507]
Biggest difference: 3.317209025457233
Smallest difference: 0.14689258966773977
Fixed eps is not success

min_eps = [0.8660254  1.65860451 0.10832327 0.8660254  0.07344629 1.62372753]
cur_eps = [1.29903811 2.48790677 0.16248491 1.29903811 0.11016944 2.4355913 ]
max_eps = [1.73205081 3.31720903 0.21664655 1.73205081 0.14689259 3.24745507]
Biggest difference: 1.6586045127286164
Smallest difference: 0.07344629483386989
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.62531182 1.24395338 0.08124244 0.47042867 0.05508471 1.21779565]
max_eps = [1.25062364 2.48790677 0.16248489 0.94085734 0.11016943 2.4355913 ]
Biggest difference: 2.4879067671417374
Smallest difference: 0.1101694254749221
Fixed eps is not success

min_eps = [0.62531182 1.24395338 0.0812

Fixed eps is not success

min_eps = [0.97993278 2.4347021  0.15901009 0.29466014 0.10781342 2.38350542]
cur_eps = [0.98771003 2.45402514 0.16027208 0.29699871 0.10866908 2.40242213]
max_eps = [0.99548727 2.47334817 0.16153406 0.29933729 0.10952474 2.42133884]
Biggest difference: 0.03864606514963276
Smallest difference: 0.0017113240693067272
Fixed eps is not success

min_eps = [0.98771003 2.45402514 0.16027208 0.29699871 0.10866908 2.40242213]
cur_eps = [0.99159865 2.46368665 0.16090307 0.298168   0.10909691 2.41188048]
max_eps = [0.99548727 2.47334817 0.16153406 0.29933729 0.10952474 2.42133884]
Biggest difference: 0.019323032574816157
Smallest difference: 0.0008556620346533567
Fixed eps is not success

min_eps = [0.99159865 2.46368665 0.16090307 0.298168   0.10909691 2.41188048]
cur_eps = [0.99354296 2.46851741 0.16121857 0.29875264 0.10931082 2.41660966]
max_eps = [0.99548727 2.47334817 0.16153406 0.29933729 0.10952474 2.42133884]
Biggest difference: 0.0096615162874083
Smallest diffe

Fixed eps is not success

min_eps = [1.75397673 2.33470919 0.99943855 0.59793383 0.13444794 0.89956298]
cur_eps = [1.78226668 2.37236579 1.01555852 0.60757792 0.13661645 0.91407206]
max_eps = [1.81055663 2.41002239 1.0316785  0.61722201 0.13878497 0.92858114]
Biggest difference: 0.0753131996648988
Smallest difference: 0.00433703031580307
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.89113333 1.14043571 0.45356303 0.30378895 0.06830821 0.3944065 ]
max_eps = [1.78226667 2.28087141 0.90712605 0.60757789 0.13661641 0.788813  ]
Biggest difference: 2.280871412403962
Smallest difference: 0.13661641406937228
Fixed eps is not success

min_eps = [0.89113333 1.14043571 0.45356303 0.30378895 0.06830821 0.3944065 ]
cur_eps = [1.3367     1.71065356 0.68034454 0.45568342 0.10246231 0.59160975]
max_eps = [1.78226667 2.28087141 0.90712605 0.60757789 0.13661641 0.788813  ]
Biggest difference: 1.140435706201981
Smallest difference: 0.06830820703468614
Fixed eps i

Fixed eps is not success

min_eps = [1.7579948  2.21439883 0.58241727 0.59930355 0.13475584 0.40875788]
cur_eps = [1.75971495 2.21656556 0.58298715 0.59988995 0.13488769 0.40915784]
max_eps = [1.7614351  2.21873229 0.58355703 0.60047635 0.13501955 0.4095578 ]
Biggest difference: 0.004333461506280667
Smallest difference: 0.0002637100585946017
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.87985748 1.09987812 0.17430675 0.29994496 0.06744377 0.09620405]
max_eps = [1.75971495 2.19975624 0.34861351 0.59988991 0.13488754 0.1924081 ]
Biggest difference: 2.1997562392825487
Smallest difference: 0.13488754485661766
Fixed eps is not success

min_eps = [0.87985748 1.09987812 0.17430675 0.29994496 0.06744377 0.09620405]
cur_eps = [1.31978621 1.64981718 0.26146013 0.44991743 0.10116566 0.14430607]
max_eps = [1.75971495 2.19975624 0.34861351 0.59988991 0.13488754 0.1924081 ]
Biggest difference: 1.0998781196412744
Smallest difference: 0.06744377242830883
Fixed

Fixed eps is not success

min_eps = [1.75864101 2.15890036 0.19857451 0.58749845 0.13480465 0.10180631]
cur_eps = [1.75874836 2.15903214 0.19858663 0.58753431 0.13481288 0.10181253]
max_eps = [1.75885572 2.15916393 0.19859875 0.58757018 0.13482111 0.10181874]
Biggest difference: 0.0002635698152291255
Smallest difference: 1.2429045774711711e-05
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.87937418 1.05882212 0.05976726 0.2728344  0.0615773  0.05090492]
max_eps = [1.75874836 2.11764424 0.11953452 0.5456688  0.12315461 0.10180985]
Biggest difference: 2.117644238274799
Smallest difference: 0.10180984669278458
Fixed eps is not success

min_eps = [0.87937418 1.05882212 0.05976726 0.2728344  0.0615773  0.05090492]
cur_eps = [1.31906127 1.58823318 0.08965089 0.4092516  0.09236595 0.07635739]
max_eps = [1.75874836 2.11764424 0.11953452 0.5456688  0.12315461 0.10180985]
Biggest difference: 1.0588221191373994
Smallest difference: 0.05090492334639229
Fixe

Fixed eps is success

min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.16796865 0.24785339 0.23221311 0.20368449 0.08780152 0.16840064
 0.02315906 0.00674727 0.00923807 0.0066971  0.00934358 0.00603712
 0.00497993 0.00903758 0.00867969]
max_eps = [0.3359373  0.49570677 0.46442623 0.40736897 0.17560304 0.33680129
 0.04631812 0.01349454 0.01847614 0.0133942  0.01868717 0.01207424
 0.00995985 0.01807517 0.01735938]
Biggest difference: 0.4957067730612228
Smallest difference: 0.009959851909833943
Fixed eps is success

min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.08398433 0.12392669 0.11610656 0.10184224 0.04390076 0.08420032
 0.01157953 0.00337363 0.00461903 0.00334855 0.00467179 0.00301856
 0.00248996 0.00451879 0.00433985]
max_eps = [0.16796865 0.24785339 0.23221311 0.20368449 0.08780152 0.16840064
 0.02315906 0.00674727 0.00923807 0.0066971  0.00934358 0.00603712
 0.00497993 0.00903758 0.00867969]
Biggest difference: 0.2478533865306114
Smalle

Fixed eps is success

min_eps = [0.00010984 0.00010587 0.05565824 0.05522078 0.01472202 0.04898128
 0.00152185 0.00638604 0.0021017 ]
cur_eps = [0.0001373  0.00013234 0.0695728  0.06902598 0.01840253 0.0612266
 0.00190231 0.00798255 0.00262713]
max_eps = [0.00016476 0.0001588  0.08348736 0.08283117 0.02208303 0.07347192
 0.00228277 0.00957906 0.00315255]
Biggest difference: 0.027829120984226345
Smallest difference: 5.293491254577376e-05
Fixed eps is not success

min_eps = [0.0001373  0.00013234 0.0695728  0.06902598 0.01840253 0.0612266
 0.00190231 0.00798255 0.00262713]
cur_eps = [0.00015103 0.00014557 0.07653008 0.07592857 0.02024278 0.06734926
 0.00209254 0.00878081 0.00288984]
max_eps = [0.00016476 0.0001588  0.08348736 0.08283117 0.02208303 0.07347192
 0.00228277 0.00957906 0.00315255]
Biggest difference: 0.013914560492113176
Smallest difference: 2.6467456272886866e-05
Fixed eps is not success

min_eps = [0.00015103 0.00014557 0.07653008 0.07592857 0.02024278 0.06734926
 0.0020925

Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.00968169 0.00947861 0.4074365  0.34166751 0.03934306 0.01082009
 0.01351319 0.04953468]
max_eps = [0.01936339 0.01895722 0.814873   0.68333501 0.07868613 0.02164018
 0.02702639 0.09906936]
Biggest difference: 0.8148729953976676
Smallest difference: 0.01895721681511824
Fixed eps is not success

min_eps = [0.00968169 0.00947861 0.4074365  0.34166751 0.03934306 0.01082009
 0.01351319 0.04953468]
cur_eps = [0.01452254 0.01421791 0.61115475 0.51250126 0.05901459 0.01623013
 0.02026979 0.07430202]
max_eps = [0.01936339 0.01895722 0.814873   0.68333501 0.07868613 0.02164018
 0.02702639 0.09906936]
Biggest difference: 0.4074364976988338
Smallest difference: 0.00947860840755912
Fixed eps is not success

min_eps = [0.01452254 0.01421791 0.61115475 0.51250126 0.05901459 0.01623013
 0.02026979 0.07430202]
cur_eps = [0.01694296 0.01658756 0.71301387 0.59791814 0.06885036 0.01893516
 0.02364809 0.08668569]
max_eps = [0.01936339

Fixed eps is not success

min_eps = [0.01088919 0.01087029 0.74849378 0.58755438 0.07548394 0.01198642
 0.02592654 0.09503767]
cur_eps = [0.01106482 0.01104561 0.76056626 0.59703106 0.07670143 0.01217975
 0.02634471 0.09657053]
max_eps = [0.01124046 0.01122094 0.77263875 0.60650775 0.07791891 0.01237308
 0.02676288 0.0981034 ]
Biggest difference: 0.024144960793537007
Smallest difference: 0.00035065443488613117
Fixed eps is not success

min_eps = [0.01106482 0.01104561 0.76056626 0.59703106 0.07670143 0.01217975
 0.02634471 0.09657053]
cur_eps = [0.01115264 0.01113328 0.76660251 0.6017694  0.07731017 0.01227642
 0.02655379 0.09733696]
max_eps = [0.01124046 0.01122094 0.77263875 0.60650775 0.07791891 0.01237308
 0.02676288 0.0981034 ]
Biggest difference: 0.012072480396768559
Smallest difference: 0.00017532721744306472
Fixed eps is not success

min_eps = [0.01115264 0.01113328 0.76660251 0.6017694  0.07731017 0.01227642
 0.02655379 0.09733696]
cur_eps = [0.01119655 0.01117711 0.76962063 0

In [ ]:
# uncomment to visualize planes (TODO)
# visualizer.region_to_collision_pair_to_plane_dictionary = iris_generator.linesearch_regions_to_certificates_by_collision_pair_map

In [ ]:
do_alt_cert = True
if do_alt_cert:
    iris_generator.certify_and_adjust_regions_by_bilinear_alternation(max_iter = 1000, 
                                                                 stopped_improving_count_threshold = 5,
                                                                 not_improving_threshold = 1e-4)

Starting Region 1/6
iter 0/1000
Fixed eps is success = True
Fixed mult program is success = True
Cur eps is 
[1.73205081 3.31720902 0.21664655 1.73205081 0.14689259 3.1431506 ]
Improvement 0.10430446575093426
iter 10/1000
Fixed eps is success = True
Fixed mult program is success = True
Cur eps is 
[0.48633229 3.30730932 0.20795123 1.03303765 0.14689259 2.63986438]
Improvement 0.43900804206224525
iter 20/1000
Fixed eps is success = True
Fixed mult program is success = True
Cur eps is 
[0.08045306 3.11130146 0.0700995  0.06110966 0.06706458 2.58305542]
Improvement 0.021629701933557725
iter 30/1000
Fixed eps is success = True
Fixed mult program is success = True
Cur eps is 
[0.05298743 2.95825861 0.0471664  0.03999263 0.04759051 2.52126601]
Improvement 0.015292494006629515
iter 40/1000
Fixed eps is success = True
Fixed mult program is success = True
Cur eps is 
[0.03346805 2.83525892 0.03018183 0.02517927 0.03150892 2.50156269]
Improvement 0.011051636160270646
iter 50/1000
Fixed eps is su

In [ ]:
# plot new regions
if do_viz:
    if do_alt_cert:
        visualizer.plot_regions(iris_generator.alternation_search_regions, region_suffix = '_new_by_alternation')
    if do_linesearch_cert:
        visualizer.plot_regions(iris_generator.linesearch_regions, region_suffix = '_new_by_linesearch')
    

In [ ]:
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra
# Solve path planning
start_time = time.time()
# trajectory through linesearch_regions
spp = BsplineTrajectoryThroughUnionOfHPolyhedra(trajectory_start.squeeze(), trajectory_end.squeeze(), 
                                                iris_generator.linesearch_regions)
spp.set_max_velocity(0.3*np.ones(plant.num_positions()))
spp.set_extra_control_points_per_region(3)
traj_iris = spp.SolveVerbose()
visualizer.draw_traj_tspace(traj_iris, 100, 'iris')

In [ ]:
print(time.time() - start_time)
print(traj_iris.start_time())
print(traj_iris.end_time())
# draw_traj_tspace(traj_iris, 100, 'iris')
#animate Iris + spp sol
substeps = 1000
its = 10
visualizer.animate_t(traj_iris, substeps, its*substeps)